# PyTorch Feature Extractor

- use <https://github.com/christiansafka/img2vec.git> to extract features with CUDA

In [1]:
%reload_ext autoreload
%autoreload 2
import os, sys
from os.path import join

import time
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
from glob import glob

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import imutils
from PIL import Image
import cv2 as cv
if not cv.__version__ == '3.4.2':
  print('pip install opencv-python==3.4.2 or greater')

In [2]:
# append notebook imports folder
sys.path.append(str(Path(os.getcwd()).parent))
from utils import imx

In [3]:
# https://github.com/christiansafka/img2vec.git

# append notebook imports folder
sys.path.append(str(Path(os.getcwd()).parent.parent/'vframe/'))

from vframe.settings import vframe_cfg as cfg
from vframe.utils import im_utils, logger_utils
from vframe.settings import types


In [4]:
# get a test image
im_test_list = glob(join(cfg.DIR_TEST_IMAGES, 'classify', '*'))
fp_im_test = np.random.choice(im_test_list)

In [5]:
opt_cuda = True
device = torch.device("cuda" if opt_cuda else "cpu")

In [6]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [7]:
#models.vgg19

In [13]:
opt_model = 'vgg19'
layer = 'default'
if opt_model == 'alexnet':
  model = models.alexnet(pretrained=True)
  if layer == 'default':
    layer = model.classifier[-2]
    layer_output_size = 4096
  else:
    layer = model.classifier[-layer]
elif opt_model == 'resnet18':
  model = models.resnet18(pretrained=True)
#   layer = model._modules.get('avgpool')
  layer = model.avgpool
  layer_output_size = 512
elif opt_model == 'vgg16':
  model = models.vgg16(pretrained=True)
  layer = model.classifier[3]
elif opt_model == 'vgg19':
  model = models.vgg19(pretrained=True)
  layer = model.classifier[3]
  layer_output_size = 4096
model = model.to(device)

In [9]:
model.parameters

<bound method Module.parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [14]:
model.classifier[3]

Linear(in_features=4096, out_features=4096, bias=True)

In [15]:
im_pil = Image.open(fp_im_test)
im_pt = normalize(to_tensor(scaler(im_pil))).unsqueeze(0).to(device)

In [19]:
#my_embedding = torch.zeros(1, layer_output_size, 1, 1)
embedding = torch.zeros(1, layer_output_size)

def copy_data(m, i, o):
  embedding.copy_(o.data)
  
h = layer.register_forward_hook(copy_data)
h_x = model(im_pt)
h_x = model(im_pt)

h.remove()

vec = my_embedding.numpy()[0, :]
vec = vec.tolist()

In [20]:
print(len(vec))
# print(tensor)

4096


In [21]:
print(vec[:10])

[-0.8514065742492676, -2.1224746704101562, 0.6155156493186951, -4.788111209869385, -1.5534257888793945, -3.008636474609375, -0.9416205883026123, -0.4355904757976532, -0.18371400237083435, -1.373375415802002]


In [22]:
vec_norm = vec/np.linalg.norm(vec)

In [23]:
print(vec_norm[:10])

[-0.00864179 -0.02154316  0.0062475  -0.04859942 -0.0157673  -0.03053772
 -0.00955747 -0.00442125 -0.0018647  -0.01393979]
